### `Participation Rate` vs `Conditional Participation Rate`.

to help understand the different summing of rates

In [2]:
#import pathlib
import json
import nzshm_model as nm
import solvis
# from solvis.fault_system_solution_helper import FaultSystemSolutionHelper
from solvis.filter import FilterSubsectionIds
from solvis.filter import FilterRuptureIds

In [146]:
# OPTIONS
SINGLE_SOLUTION = True
SECTION_RATE = False # otherwise Parent rates

if SINGLE_SOLUTION:
    solution = solvis.InversionSolution.from_archive("NZSHM22_ScaledInversionSolution-QXV0b21hdGlvblRhc2s6MTEzMTM0.zip")
else:
    current_model = nm.get_model_version("NSHM_v1.0.4")
    slt = current_model.source_logic_tree
    csol = solvis.CompositeSolution.from_archive("NSHM_v1.0.4_CompositeSolution.zip", slt)
    solution = csol._solutions['CRU']

# helper = FaultSystemSolutionHelper(solution)  
rate_column = "Annual Rate" if isinstance(solution, solvis.InversionSolution) else "rate_weighted_mean"


In [147]:
# get ruptures passing through a named parent fault
TARGET_FAULTS = ['Opotiki 3', 'Calypso 2']
# TARGET_FAULTS = ['Opotiki 3'] 

# ruptures = helper.ruptures_for_parent_fault_names(TARGET_FAULTS) # 
ruptures = FilterRuptureIds(solution)\
    .for_parent_fault_names(TARGET_FAULTS)
# # get rupture fault sections (rs) with rates for those ruptures
# df0 = solution.rs_with_rupture_rates
# rupture_sections_df = df0[df0["Rupture Index"].isin(ruptures)]

### calculate participation rate series

In [148]:
# for comparison
opo_section = 1426 # Opotiki 3, Subsection 2
cal_section = 217  # Calypso 2, Subsection 1
df0 = solution.rs_with_rupture_rates.copy()

print("summing the subsection rate")
print("===========================")

print("new sum() filtering by section ID: `Participation Rate` (all ruptures in the system)")
print(f'Opotiki rate: {df0[df0.section==opo_section][rate_column].sum()}')
print(f'Calypso rate: {df0[df0.section==cal_section][rate_column].sum()}')
print()
print("API sum() filtering by rupture ID: `Conditional Participation Rate` (filtered by rupture id or attribute)")
df0 = df0[df0["Rupture Index"].isin(ruptures)]
print(f'Opotiki rate: {df0[df0.section==opo_section][rate_column].sum()}')
print(f'Calypso rate: {df0[df0.section==cal_section][rate_column].sum()}')

summing the subsection rate
new sum() filtering by section ID: `Participation Rate` (all ruptures in the system)
Opotiki rate: 0.0001605307770660147
Calypso rate: 0.0002327073598280549

API sum() filtering by rupture ID: `Conditional Participation Rate` (filtered by rupture id or attribute)
Opotiki rate: 0.0001605307770660147
Calypso rate: 0.0002327073598280549


In [93]:
def participation_rate(solution: solvis.inversion_solution.typing.InversionSolutionProtocol, section_ids, rupture_ids=None):
    rate_column = "Annual Rate" if isinstance(solution, solvis.InversionSolution) else "rate_weighted_mean"
    # df0 = solution.ruptures_with_rupture_rates[["Rupture Index", "Annual Rate"]]
    df0 = solution.rs_with_rupture_rates
    df0 = df0[df0["section"].isin(section_ids)]
    if rupture_ids:
        print('CONDITIONAL')
        df0 = df0[df0["Rupture Index"].isin(ruptures)]
    else:
        print('UNCONDITIONAL')
    return df0.pivot_table(
       values=rate_column, index=['section'], aggfunc='sum'
    )

In [96]:
# csol = solvis.CompositeSolution.from_archive("NSHM_v1.0.4_CompositeSolution.zip", slt)
# solution = csol._solutions['CRU']

# helper = FaultSystemSolutionHelper(solution)  
rate_column = "Annual Rate" if isinstance(solution, solvis.InversionSolution) else "rate_weighted_mean"

sections = [opo_section, cal_section]
print(participation_rate(solution, sections))

# csol = solvis.CompositeSolution.from_archive("NSHM_v1.0.4_CompositeSolution.zip", slt)
# solution = csol._solutions['CRU']
print(participation_rate(solution, sections, ruptures))

UNCONDITIONAL
         rate_weighted_mean
section                    
217.0              0.000417
1426.0             0.000172
CONDITIONAL
         rate_weighted_mean
section                    
217.0              0.000017
1426.0             0.000172


In [168]:
from typing import Dict, Iterator, List, Iterable, Optional
from solvis.filter import FilterSubsectionIds, FilterRuptureIds
from solvis.inversion_solution import InversionSolution
from solvis.inversion_solution.typing import InversionSolutionProtocol

def section_participation_rates(
    solution: InversionSolutionProtocol,
    section_ids: Iterable[int],
    rupture_ids: Optional[Iterable[int]]=None):

    rate_column = "Annual Rate" if isinstance(solution, InversionSolution) else "rate_weighted_mean"

    df0 = solution.rs_with_rupture_rates
    df0 = df0[df0["section"].isin(section_ids)]
    if rupture_ids:
        df0 = df0[df0["Rupture Index"].isin(rupture_ids)]
    return df0.pivot_table(
       values=rate_column, index=['section'], aggfunc='sum'
    )

def fault_participation_rates(
    solution: InversionSolution,
    fault_names: Iterable[str],
    rupture_ids: Optional[Iterable[int]]=None):
    """
    get the 'participation rate" of a given parent fault.

    That is, the sum of rates for all ruptures that involve the requested parent fault .
    """
    subsection_ids = FilterSubsectionIds(solution).for_parent_fault_names(fault_names)

    df0 = solution.rs_with_rupture_rates
    df0 = df0[df0["section"].isin(subsection_ids)]
    if rupture_ids:
        df0 = df0[df0["Rupture Index"].isin(rupture_ids)]
    
    df1 = df0.join(solution.fault_sections[['ParentID','ParentName']], on='section')    
    # return df1
    return df1.pivot_table(
       values=rate_column, index='ParentID', aggfunc='sum'
    )   
    # return section_participation_rates(solution, subsection_ids, rupture_ids)


In [176]:
TARGET_FAULTS = ['Opotiki 2', 'Opotiki 3', 'Calypso 2']
ruptures = FilterRuptureIds(solution)\
    .for_parent_fault_names(TARGET_FAULTS)\
    .for_magnitude(min_mag=7.8, max_mag=8.3)

df0 = fault_participation_rates(solution, TARGET_FAULTS)
df0

solution.fault_sections.join(df0, on='ParentID', how='inner')
# df1 = df0.join(solution.fault_sections[['ParentID','ParentName']], on='ParentID', how='right')
# df1
# df0.join(solution.fault_sections[['ParentID','ParentName']], on='section')

,FaultID,FaultName,DipDeg,Rake,LowDepth,UpDepth,DipDir,AseismicSlipFactor,CouplingCoeff,ParentID,ParentName,geometry,Target Slip Rate,Target Slip Rate StdDev,Annual Rate
216,216,"Calypso 2, Subsection 0",50.0,-90.0,14.3405,0.0,318.1,0.0,1.0,52,Calypso 2,"LINESTRING (177.0853 -37.6354, 177.14271 -37.5...",0.001610,0.000303,0.000424
217,217,"Calypso 2, Subsection 1",50.0,-90.0,14.3405,0.0,318.1,0.0,1.0,52,Calypso 2,"LINESTRING (177.14271 -37.59376, 177.1589 -37....",0.001610,0.000303,0.000424
1422,1422,"Opotiki 2, Subsection 0",70.0,-110.0,24.0000,0.0,97.9,0.0,1.0,329,Opotiki 2,"LINESTRING (177.2775 -37.9693, 177.2876 -37.93...",0.001058,0.000634,0.000064
1423,1423,"Opotiki 2, Subsection 1",70.0,-110.0,24.0000,0.0,97.9,0.0,1.0,329,Opotiki 2,"LINESTRING (177.28082 -37.90698, 177.2749 -37....",0.001058,0.000634,0.000064
1424,1424,"Opotiki 3, Subsection 0",70.0,-110.0,19.2000,0.0,284.5,0.0,1.0,330,Opotiki 3,"LINESTRING (177.2411 -37.9645, 177.2644 -37.92...",0.001935,0.000455,0.000515
1425,1425,"Opotiki 3, Subsection 1",70.0,-110.0,19.2000,0.0,284.5,0.0,1.0,330,Opotiki 3,"LINESTRING (177.26486 -37.89503, 177.2651 -37....",0.002407,0.000455,0.000515
1426,1426,"Opotiki 3, Subsection 2",70.0,-110.0,19.2000,0.0,284.5,0.0,1.0,330,Opotiki 3,"LINESTRING (177.28504 -37.82331, 177.3115 -37....",0.001597,0.000508,0.000515
1427,1427,"Opotiki 3, Subsection 3",70.0,-110.0,19.2000,0.0,284.5,0.0,1.0,330,Opotiki 3,"LINESTRING (177.3115 -37.75232, 177.3252 -37.7...",0.001295,0.000504,0.000515


In [131]:
df0 = fault_participation_rates(solution, TARGET_FAULTS, ruptures)
df1 = df0.join(solution.fault_sections[['ParentID','ParentName']], on='section')
df1

,rate_weighted_mean,ParentID,ParentName
section,,,
216.0,0.000003,52,Calypso 2
217.0,0.000003,52,Calypso 2
1424.0,0.000085,330,Opotiki 3
1425.0,0.000085,330,Opotiki 3
1426.0,0.000060,330,Opotiki 3
1427.0,0.000059,330,Opotiki 3


In [97]:
## same for parent fault summation

opo_fault = 330 # Opotiki 3
cal_fault = 52  # Calypso 2, Subsection 1

df0 = solution.rs_with_rupture_rates
df1 = df0.join(solution.fault_sections[['ParentID']], on='section')
# df1[df1.ParentID==cal_fault]
# assert 0

df2 = df1[["Rupture Index", "ParentID", "section", rate_column]]
# parent_rates = fault_rates.groupby("ParentID").agg('sum')[rate_column]

print("summing the parent fault rate")
print("========================--===")

print("new sum() filtering by section ID: `Participation Rate` (all ruptures in the system)")
print(f'Opotiki rate: {df2[df2.ParentID==opo_fault][rate_column].sum()}')
print(f'Calypso rate: {df2[df2.ParentID==cal_fault][rate_column].sum()}')
print()
print("API sum() filtering by rupture ID: `Conditional Participation Rate` (filtered by rupture id or attribute)")
df3 = df2[df2["Rupture Index"].isin(ruptures)]
print(f'Opotiki rate: {df3[df3.ParentID==opo_fault][rate_column].sum()}')
print(f'Calypso rate: {df3[df3.ParentID==cal_fault][rate_column].sum()}')


summing the parent fault rate
========================--===
new sum() filtering by section ID: `Participation Rate` (all ruptures in the system)
Opotiki rate: 0.0005705734991546631
Calypso rate: 0.0008173589876243428

API sum() filtering by rupture ID: `Conditional Participation Rate` (filtered by rupture id or attribute)
Opotiki rate: 0.0005705734991546631
Calypso rate: 1.6794373451884402e-05
